
# Rask API — Notebook minimal
Ce carnet contient trois blocs utiles et polis :
1. **Récupérer un token** OAuth2 avec `client_id` et `client_secret`  
2. **Retrouver l'ID d'un projet** à partir de son **nom**  
3. **Télécharger la transcription** et produire un **DataFrame** exploitable (avec durée en secondes)


In [ ]:

# (1) Dépendances — exécuter une seule fois si nécessaire
# Astuce: si vous avez déjà ces libs dans votre kernel, cette cellule ne sert à rien.
%pip install --quiet requests pandas python-dotenv


In [ ]:

# (2) Récupération du token à partir des accès client (client_id / client_secret)
# - Priorité aux variables d'environnement (conseillé): RASK_CLIENT_ID / RASK_CLIENT_SECRET
# - Sinon, vous pouvez les renseigner directement ci-dessous
import os
import requests
from dotenv import load_dotenv

load_dotenv()

TOKEN_URL = "https://rask-prod.auth.us-east-2.amazoncognito.com/oauth2/token"
SCOPES = "api/source api/input api/output api/limit"

# Option A: variables d'environnement
CLIENT_ID = os.getenv("RASK_CLIENT_ID") or ""
CLIENT_SECRET = os.getenv("RASK_CLIENT_SECRET") or ""

# Option B: définition manuelle (remplacez par vos valeurs si besoin)
# CLIENT_ID = "votre_client_id"
# CLIENT_SECRET = "votre_client_secret"

def get_token(client_id: str, client_secret: str) -> str:
    """Retourne un access_token OAuth2 (client-credentials)."""
    if not client_id or not client_secret:
        raise RuntimeError("CLIENT_ID / CLIENT_SECRET manquants. Définissez RASK_CLIENT_ID et RASK_CLIENT_SECRET ou renseignez-les ici.")
    r = requests.post(
        TOKEN_URL,
        data={
            "grant_type": "client_credentials",
            "scope": SCOPES,
        },
        auth=(client_id, client_secret),
        timeout=30,
    )
    r.raise_for_status()
    return r.json()["access_token"]

TOKEN = get_token(CLIENT_ID, CLIENT_SECRET)
HEADERS = {"Authorization": f"Bearer {TOKEN}"}

print("Token récupéré avec succès. (prefix) ->", TOKEN[:12] + "…")


In [ ]:

# (3) Trouver l'ID du projet par son nom
# La recherche scanne la liste paginée des projets et fait une normalisation simple du nom.
import re
import requests

PROJECTS_URL = "https://api.rask.ai/v2/projects"

def _normalize_name(s: str) -> str:
    """Normalise : retire espaces, underscores et tirets, passe en minuscules."""
    return re.sub(r"[-\s_]+", "", (s or "").strip().lower())

def find_project_id_by_name(headers: dict, name: str, page_size: int = 100) -> str | None:
    target = _normalize_name(name)
    offset = 0
    # Premier passage: correspondance exacte normalisée
    while True:
        r = requests.get(PROJECTS_URL, headers=headers, params={"offset": offset, "limit": page_size}, timeout=30)
        r.raise_for_status()
        payload = r.json()
        projects = payload.get("projects", [])
        for p in projects:
            if _normalize_name(p.get("name")) == target:
                return p["id"]
        total = payload.get("total", 0)
        offset += page_size
        if offset >= total:
            break
    # Second passage: nom cible inclus (fallback)
    offset = 0
    while True:
        r = requests.get(PROJECTS_URL, headers=headers, params={"offset": offset, "limit": page_size}, timeout=30)
        r.raise_for_status()
        payload = r.json()
        projects = payload.get("projects", [])
        for p in projects:
            if target in _normalize_name(p.get("name")):
                return p["id"]
        total = payload.get("total", 0)
        offset += page_size
        if offset >= total:
            break
    return None

# EXEMPLE D'UTILISATION (décommentez et remplacez par votre nom de projet):
# project_name = "1_-_Introduction_et_presentation_du_materiels_KtwWjO_m8phbqY"
# project_id = find_project_id_by_name(HEADERS, project_name)
# print("Project ID ->", project_id)


In [ ]:

# (4) Récupérer la transcription et en faire un DataFrame
# On calcule aussi la durée en secondes par segment.
import math
import pandas as pd
import re
import requests

TRANSCRIPTION_URL = "https://api.rask.ai/v2/projects/{project_id}/transcription"

def _parse_timecode(tc) -> float:
    """Accepte HH:MM:SS,mmm | HH:MM:SS.mmm | secondes (float/int). Retourne des secondes (float)."""
    if isinstance(tc, (int, float)): 
        return float(tc)
    s = str(tc).strip().replace(",", ".")
    m = re.match(r"^(?P<h>\d{1,2}):(?P<m>\d{2}):(?P<s>\d{2})(?:\.(?P<ms>\d{1,3}))?$", s)
    if m:
        h = int(m.group("h")); mn = int(m.group("m")); sec = int(m.group("s")); ms = int(m.group("ms") or 0)
        return h*3600 + mn*60 + sec + ms/1000.0
    return float(s)

def get_transcription_df(headers: dict, project_id: str) -> pd.DataFrame:
    """Télécharge la transcription et renvoie un DataFrame avec start, end, duration_sec, src_text, dst_text."""
    r = requests.get(TRANSCRIPTION_URL.format(project_id=project_id), headers=headers, timeout=60)
    r.raise_for_status()
    data = r.json()
    rows = []
    for seg in data.get("segments", []):
        start = seg.get("start"); end = seg.get("end")
        dur = max(0.0, _parse_timecode(end) - _parse_timecode(start))
        src = (seg.get("src") or {}).get("text", "")
        dst = (seg.get("dst") or {}).get("text", "")
        rows.append({
            "segment_id": seg.get("id"),
            "speaker": seg.get("speaker"),
            "start": start,
            "end": end,
            "duration_sec": round(dur, 3),
            "src_text": src,
            "dst_text": dst,
        })
    return pd.DataFrame(rows)

# EXEMPLE D'UTILISATION (décommentez pour tester):
# project_name = "1_-_Introduction_et_presentation_du_materiels_KtwWjO_m8phbqY"
# project_id = find_project_id_by_name(HEADERS, project_name)
# df = get_transcription_df(HEADERS, project_id)
# df.head(10)
